## Import das bibliotecas necessárias
    

In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import math
import time

import global_
import results
import get
import functions
import cfg

# **Configuração Inicial**

Insira o caminho do vídeo aqui:

In [2]:
path = "D:/Documentos/Unicamp/IC/Videos/"
folder = ""
name = "basal"
extension = ".avi"

Exemplo:

`path: "C:/Videos/"`

`name: "meu_video"`

`extension: ".avi"`


Coloque a distância entre dois pixels da imagem **em μm**

In [3]:
d_pixel = 3.5 # Distancia entre pixel = 3.5 um

Coloque o peso do inseto **em miligramas**

In [4]:
weight = 120.1 # Peso = 120.1 mg

Determine se a detecção de sístole e diástole será feita através do gráfico da área do coração ou através da área da elipse

| Método de detecção | Descrição |
| --- | --- |
| 1 | Detecção de frequencia cardíaca através da análise da área do contorno do coração |
| 2 | Detecção de frequencia cardíaca através da análise da área da elipse |

In [5]:
detection_method = 1  # 1 = Detecção por area do contorno do coracao

# **Funcionamento**

Depois de configurar as variáveis iniciais na seção acima, execute o notebook.

Serão apresentados duas telas iniciais

The results will be presented at the end of the Notebook at section [Results](#results)

## **1 Funções Principais**

###  Ellipse

In [6]:
def ellipse_rotation(args):
    global ellipse_angle, rec
    global ellipse_center, ellipse_height, ellipse_width, ellipse_size
    global ellipse_vertex

    drawing = np.copy(cache)

    ellipse_angle = args

    rec = rectangle_ellipse((ellipse_center,ellipse_size,ellipse_angle))

    cv.circle(drawing,(ellipse_vertex[0][0],ellipse_vertex[0][1]),0,(0,0,255),8) # Ponto de baixo da elipse
    cv.circle(drawing,(ellipse_vertex[1][0],ellipse_vertex[1][1]),0,(0,0,255),8) # Ponto de cima da elipse
    cv.circle(drawing,(ellipse_vertex[2][0],ellipse_vertex[2][1]),0,(0,0,255),8) # Ponto lateral direito da elipse
    cv.circle(drawing,(ellipse_vertex[3][0],ellipse_vertex[3][1]),0,(0,0,255),8) # Ponto lateral esquerdo da elips
    cv.circle(drawing,(ellipse_center[0],ellipse_center[1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse

    cv.ellipse(drawing,(ellipse_center,ellipse_size,ellipse_angle), (90,90,255), 1)

    cv.imshow('Original',drawing)   
    
    return

In [7]:
def get_paramaters2(d_px,c,x_ax,y_ax):

    left_x = np.array([])
    right_x = np.array([])

    interval = 5

    for i in range (len(c)):
        if abs(c[i][0][1] - y_ax) <= 5:
            if c[i][0][0] <= x_ax:
                left_x = np.append(left_x,x_ax - c[i][0][0])
            else:
                right_x = np.append(right_x,c[i][0][0] - x_ax)

    topmost = tuple(c[c[:,:,1].argmin()][0])
    bottommost = tuple(c[c[:,:,1].argmax()][0])

    b = np.average(left_x) + np.average(right_x)
    a = bottommost[1] - topmost[1]
   
    volume = int((1/6)*np.pi*a*b**2) 

    return int(a*d_px), int(b*d_px) ,int(volume*d_px**3)

###  Contour

In [8]:
def find_largest_contour (cont,centers,elipses):
    for i, c in enumerate(cont):

        top_left_x,top_left_y,largura,altura = cv.boundingRect(c)

        centerx = (2*top_left_x + largura)//2
        centery = (2*top_left_y + altura)//2

        centers.append([centerx,centery])

        if cv.contourArea(c) > max_area: 
            max_area = cv.contourArea(c)
            estimated_area = max_area
            estimeated_height = altura
            estimated_width = largura
            estimated_top_left_x = top_left_x
            estimated_top_left_y = top_left_y
            estimated_centerx = centerx
            estimated_centery = centery
            best_i = i

            try:
                elipses[i] = cv.fitEllipseDirect(c)
            except:
                None

def remove_division(cont,main_i):

    joined_cont = cont
    joined_i = list()

    x_main_cont = cont[main_i][0][0][0]
    y_main_cont = cont[main_i][0][0][1]

    tmp_draw = np.zeros((frame.shape[0], frame.shape[1], 4))
    try_i = 0
    min_dist = 0
    
    removed = False

    while True:
        cv.imshow('Juntar', tmp_draw)

        cmd = cv.waitKey(0)

        if cmd == ord('n'):

            cont_temp = joined_cont
            
            tmp_draw = np.zeros((frame.shape[0], frame.shape[1], 4))

            try_i = (try_i + 1)%len(cont)

            if (try_i == main_i):
                try_i = (try_i + 1)%len(cont)

            s = set(joined_i)

            if (len(joined_cont)!=0) and (try_i in s):
                try_i = (try_i + 1)%len(cont)

            
            cont_temp = join_countours(cont_temp,main_i,try_i)

            cv.drawContours(tmp_draw, cont_temp, try_i, (255,255,255))
            cv.drawContours(tmp_draw, cont, main_i, (0,255,255))
            
            removed = True
            
        elif cmd == ord('j') and removed:
            joined_cont = cont_temp
            joined_i.append(try_i)
            tmp_draw = np.zeros((frame.shape[0], frame.shape[1], 4))
            cv.drawContours(tmp_draw, cont_temp, main_i, (0,255,255))

        elif cmd == ord('c') and removed:
            cv.destroyWindow('Juntar')
            break

        elif cmd == ord('s'):
            cv.destroyWindow('Juntar')        
            cont_temp = cont
            break              



    return cont_temp


def join_countours(c,main_i,min_i):

    c = np.copy(c)
    
    x_main_cont = c[main_i][0][0][0]
    y_main_cont = c[main_i][0][0][1]

    x_min_cont = c[min_i][0][0][0]
    y_min_cont = c[min_i][0][0][1]

    min_lenght = c[min_i].shape[0]
    main_lenght = c[main_i].shape[0]

    if y_min_cont < y_main_cont :
        for i in range(len(c[min_i])):

            x_cont = c[min_i][i][0][0]
            y_cont = c[min_i][i][0][1]

            dist = distance(x_cont,y_cont,x_main_cont,y_main_cont)

            if i == 0:
                min_dist = dist
                cont_i = i
            else:
                if (dist < min_dist):
                    cont_i = i
                    min_dist = dist

        for i in range(len(c[main_i])):
            c[min_i] = np.insert(c[min_i], 2*cont_i + 2*(i + 1), [c[main_i][i][0][0],c[main_i][i][0][1]])
        
        c[min_i] = np.reshape(c[min_i],(min_lenght + main_lenght,1,2))
        
    elif y_min_cont > y_main_cont:
        for i in range(len(c[main_i])):

            x_cont = c[main_i][i][0][0]
            y_cont = c[main_i][i][0][1]

            dist = distance(x_cont,y_cont,x_min_cont,y_min_cont)

            if i == 0:
                min_dist = dist
                cont_i = i
            else:
                if (dist < min_dist):
                    cont_i = i
                    min_dist = dist
        for i in range(len(c[min_i])):
            c[main_i] = np.insert(c[main_i], 2*cont_i + 2*(i + 1), [c[min_i][i][0][0],c[min_i][i][0][1]])

        c[main_i] = np.reshape(c[main_i],(min_lenght + main_lenght,1,2))
    return c

###  Função video_progression

In [9]:
def video_progression(args):

    global skip
    
    if key != ord('n') or key != ord('n'):

        next_frame = video.get(cv.CAP_PROP_POS_FRAMES)
        current_frame = next_frame - 1
        previous_frame = current_frame - 1

        if args < current_frame:
            if previous_frame >= 0:
                video.set(cv.CAP_PROP_POS_FRAMES, previous_frame)
            else:
                previous_frame = 0
    
        skip = True

    return


###  Função modify_ellipse

In [10]:
def modify_elipse(action, x, y, flags, parameters):
    
    frame = np.copy(parameters[0])
    #rec = parameters[1]
    cache = np.copy(parameters[2])
    d_pixel = parameters[3]

    import global_

    global edit
    # Declara o vetor com os pontos da elipse como global
    global ellipse_vertex, ellipse_angle, rec

    global ellipse_center, ellipse_height, ellipse_width, ellipse_size

    
    if action == cv.EVENT_LBUTTONDOWN:

#cv.ellipse(drawing,(ellipse_center,ellipse_size,ellipse_angle), (90,90,255), 1)

        cv.ellipse(cache,global_.ellipse_param, (0,0,255), 1)
        
        if (((x> global_.ellipse_vertex[3][0]) and (x<global_.ellipse_vertex[2][0]) and (y>global_.ellipse_vertex[1][1]) and (y<global_.ellipse_vertex[0][1])
            and (global_.ellipse_angle <=90)) or ((x<global_.ellipse_vertex[3][0]) and (x>global_.ellipse_vertex[2][0]) and (y<global_.ellipse_vertex[1][1]) 
            and (y>global_.ellipse_vertex[0][1]) and (global_.ellipse_angle >90))) :

            cv.circle(cache,(global_.ellipse_vertex[0][0],global_.ellipse_vertex[0][1]),0,(0,0,255),8) # Ponto de baixo da elipse
            cv.circle(cache,(global_.ellipse_vertex[1][0],global_.ellipse_vertex[1][1]),0,(0,0,255),8) # Ponto de cima da elipse
            cv.circle(cache,(global_.ellipse_vertex[2][0],global_.ellipse_vertex[2][1]),0,(0,0,255),8) # Ponto lateral direito da elipse
            cv.circle(cache,(global_.ellipse_vertex[3][0],global_.ellipse_vertex[3][1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse
            cv.circle(cache,(global_.ellipse_center[0],global_.ellipse_center[1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse

            cv.circle(cache,(rec[0][0],rec[0][1]),0,(0,0,255),8) # Canto inferior esquerdo do retângulo
            cv.circle(cache,(rec[1][0],rec[1][1]),0,(0,0,255),8) # Canto superior esquerdo do retângulo
            cv.circle(cache,(rec[2][0],rec[2][1]),0,(0,0,255),8) # Canto superior direito do retângulo
            cv.circle(cache,(rec[3][0],rec[3][1]),0,(0,0,255),8) # Canto inferior direito do retângulo
            # Desenha o retângulo que envolve a elipse
            box = np.array([[rec[0][0],rec[0][1]],[rec[1][0],rec[1][1]],[rec[2][0],rec[2][1]],[rec[3][0],rec[3][1]]])
            cv.drawContours(cache,[box],0,(0,0,255),1)
            cv.imshow('Original',cache)
            edit = True
        else:
            cv.imshow('Original',cache)
            edit = False


    elif (action == cv.EVENT_MOUSEMOVE) and (flags == cv.EVENT_FLAG_LBUTTON):

        #cv.circle(cache,(vertex_ellipse[0][0],vertex_ellipse[0][1]),0,(0,0,255),8) # Ponto de baixo da elipse
        #cv.circle(cache,(vertex_ellipse[1][0],vertex_ellipse[1][1]),0,(0,255,255),8) # Ponto de cima da elipse
        #cv.circle(cache,(vertex_ellipse[2][0],vertex_ellipse[2][1]),0,(0,0,255),8) # Ponto lateral direito da elipse
        #cv.circle(cache,(vertex_ellipse[3][0],vertex_ellipse[3][1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse

        #cv.circle(cache,(rec[0][0],rec[0][1]),0,(0,0,255),8) # Canto inferior esquerdo do retângulo
        #cv.circle(cache,(rec[1][0],rec[1][1]),0,(0,0,255),8) # Canto superior esquerdo do retângulo
        #cv.circle(cache,(rec[2][0],rec[2][1]),0,(0,0,255),8) # Canto superior direito do retângulo
        #cv.circle(cache,(rec[3][0],rec[3][1]),0,(0,0,255),8) # Canto inferior direito do retângulo

        
        #cv.rectangle(cache,(rec[0][0],rec[0][1]),(rec[2][0],rec[2][1]),(0,255,0),1)

        # Redimensionamento da parte de baixo da elipse
        if functions.distance(global_.ellipse_vertex[0][0],global_.ellipse_vertex[0][1],x,y) < 10:
            dif = global_.ellipse_vertex[0][1] - y
            global_.ellipse_vertex[0][1] =  y
            rec[0][1] -= dif
            rec[3][1] -= dif            
        
        # Redimensionamento da parte de cima da elipse
        elif functions.distance(global_.ellipse_vertex[1][0],global_.ellipse_vertex[1][1],x,y) < 10:
            dif = global_.ellipse_vertex[1][1] - y
            global_.ellipse_vertex[1][1] =  y
            rec[1][1] -= dif
            rec[2][1] -= dif

        # Redimensionamento da parte lateral direita da elipse
        elif functions.distance(global_.ellipse_vertex[2][0],global_.ellipse_vertex[2][1],x,y) < 10:
            dif = global_.ellipse_vertex[2][0] - x
            global_.ellipse_vertex[2][0] =  x
            rec[2][0] -= dif
            rec[3][0] -= dif
        
        # Redimensionamento da parte lateral esquerdo da elipse
        elif functions.distance(global_.ellipse_vertex[3][0],global_.ellipse_vertex[3][1],x,y) < 10:
            dif = global_.ellipse_vertex[3][0] - x
            global_.ellipse_vertex[3][0] =  x
            rec[0][0] -= dif
            rec[1][0] -= dif


        # Move a elipse
        elif functions.distance(global_.ellipse_center[0],global_.ellipse_center[1],x,y) < 10:
            difx = global_.ellipse_center[0] - x
            dify = global_.ellipse_center[1] - y
            #ellipse_vertex[3][0] =  - x
            rec[0][0] -= difx
            rec[1][0] -= difx
            rec[2][0] -= difx
            rec[3][0] -= difx


            rec[1][1] -= dify
            rec[2][1] -= dify
            rec[0][1] -= dify
            rec[3][1] -= dify            

        #get_ellipse_param()
        #rectangle_ellipse((ellipse_center,ellipse_size,ellipse_angle))

        # Calcula a posição do ponto lateral direito e esquerdo da elipse
        global_.ellipse_vertex[2][1] = (rec[2][1] + rec[3][1])//2
        global_.ellipse_vertex[3][1] = (rec[0][1] + rec[1][1])//2
        
        # Calcula a posição do ponto superior e inferior da elipse
        global_.ellipse_vertex[1][0] = (rec[1][0] + rec[2][0])//2
        global_.ellipse_vertex[0][0] = (rec[0][0] + rec[3][0])//2

        global_.ellipse_vertex[2][0] = (rec[2][0] + rec[3][0])//2
        global_.ellipse_vertex[3][0] = (rec[0][0] + rec[1][0])//2
        global_.ellipse_vertex[1][1] = (rec[1][1] + rec[2][1])//2
        global_.ellipse_vertex[0][1] = (rec[0][1] + rec[3][1])//2

        get.ellipse_paramaters(d_pixel)

        # Desenha o retângulo que envolve a elipse
        #box = np.array([[rec[0][0],rec[0][1]],[rec[1][0],rec[1][1]],[rec[2][0],rec[2][1]],[rec[3][0],rec[3][1]]])
        #cv.drawContours(cache,[box],0,(0,0,255),1)

        # Desenha a elipse
        cv.ellipse(cache,global_.ellipse_param, (90,90,255), 1)

        #cv.ellipse(cache, box, color = (0, 0, 255), thickness = 1)

        cv.imshow('Original',cache)

        edit = True

        #height,width,volume = get_paramaters(d_pixel)

        #partial_info  = np.copy(info_frame)
        #cv.putText(partial_info,"Volume: " + str(int(volume)/1e+9) + " ul", (30,70),font ,1/2,font_color, 2) 
        #cv.imshow('Informacao',partial_info)

        #cv.circle(frame,(x,y),0,(0,255,255),8)
    
    elif (action == cv.EVENT_LBUTTONUP):

        if edit == True:
            print(3.5*np.sqrt((global_.ellipse_vertex[0][0] - global_.ellipse_vertex[1][0])**2 + (global_.ellipse_vertex[0][1] - global_.ellipse_vertex[1][1])**2))
            edit = False
            stop = False
            try:

                rec = functions.rectangle_ellipse((global_.ellipse_center,global_.ellipse_size,global_.ellipse_angle))
                
                cv.circle(cache,(global_.ellipse_vertex[0][0],global_.ellipse_vertex[0][1]),0,(0,0,255),8) # Ponto de baixo da elipse
                cv.circle(cache,(global_.ellipse_vertex[1][0],global_.ellipse_vertex[1][1]),0,(0,0,255),8) # Ponto de cima da elipse
                cv.circle(cache,(global_.ellipse_vertex[2][0],global_.ellipse_vertex[2][1]),0,(0,0,255),8) # Ponto lateral direito da elipse
                cv.circle(cache,(global_.ellipse_vertex[3][0],global_.ellipse_vertex[3][1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse
                cv.circle(cache,(global_.ellipse_center[0],global_.ellipse_center[1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse

                cv.circle(cache,(rec[0][0],rec[0][1]),0,(0,0,255),8) # Canto inferior esquerdo do retângulo
                cv.circle(cache,(rec[1][0],rec[1][1]),0,(0,0,255),8) # Canto superior esquerdo do retângulo
                cv.circle(cache,(rec[2][0],rec[2][1]),0,(0,0,255),8) # Canto superior direito do retângulo
                cv.circle(cache,(rec[3][0],rec[3][1]),0,(0,0,255),8) # Canto inferior direito do retângulo
                # Desenha o retângulo que envolve a elipse
                box = np.array([[rec[0][0],rec[0][1]],[rec[1][0],rec[1][1]],[rec[2][0],rec[2][1]],[rec[3][0],rec[3][1]]])
                cv.drawContours(cache,[box],0,(0,0,255),1)
                cv.ellipse(cache,global_.ellipse_param, (0,0,255), 1)
                cv.imshow('Original',cache)
            except:
                None

        
        

    return

## **1.1 Definições de Funções Auxiliares**

In [11]:
def aux_plot_points(v):
    
    fig, ax = plt.subplots()

    for i in range (len(v)):

        x = list()
        y = list()

        for j in range(len(v[i])):
            x.append(frame_width - v[i][j][0][0])
            y.append(frame_height - v[i][j][0][1])
            
        ax.plot(x, y, 'o--' ,color='grey', alpha=0.3)
    
    plt.show()

    #plot_img(x,y,"sem","Sem", "Contorno", frame_height//50, frame_width//50, True, 0, frame_width, 0, frame_height)

    return

In [12]:
def plot_img(x_axis, y_axis, x_title, y_title, plot_title, h,w, axis, xmin, xmax, ymin, ymax, minimum, maximum):
   fig = plt.figure()  # an empty figure with no Axes
   fig, ax = plt.subplots(figsize=(h, w))
   ax.plot(x_axis, y_axis)  # Plot some data on the axes.

   if axis:
      ax.set(xlim=(xmin, xmax), ylim=(ymin, ymax))


   ax.set(xlabel= x_title, ylabel= y_title,
         title= plot_title)
         
   ax.grid() 
   
   for i in range(len(min_list)):

      ax.annotate('MIN', xy=(x_axis[minimum[i]], y_axis[minimum[i]]), arrowprops=dict(facecolor='black', shrink=0.05))
   
   for i in range(len(max_list)):

      ax.annotate('MAX', xy=(x_axis[int(maximum[i])], y_axis[int(maximum[i])]), arrowprops=dict(facecolor='black', shrink=0.05))

In [13]:
def save_video(video_array, video_name):
    out = cv.VideoWriter(video_name,cv.VideoWriter_fourcc('M','J','P','G'), fps, frame_size)
    
    for i in range(len(video_array)):
        out.write(video_array[i])
    out.release()

    return

In [14]:
def distance(x1,y1,x2,y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

# **Programa principal**

In [15]:
# open video
file = path + folder + name + extension

# Get 3 main parameters trought user's configuration
# Binary treshold = Binary treshold to detect heart's contour
# Axis rotation = Location of the axis to calculate the revolution solid
# Scale = Horizontal scale applied on the contour
binary_treshold, rot_axis_loc, scale, hor_axis = get.filters_parameters(file)

print("Trehold:",binary_treshold)
print("Axis:", rot_axis_loc)
print("Horrizontal Scale:", scale)
print("Horrizontal Axis:", hor_axis)

global_.avg_x_center, global_.avg_y_center = get.average_center(file,binary_treshold)

video = cv.VideoCapture(file)   # Open video

# Test if video was correctly open
if video.isOpened: 
    print("Was open correctly")
    
    # get video fps and number of frames
    fps = int(video.get(cv.CAP_PROP_FPS))
    total_frame = video.get(cv.CAP_PROP_FRAME_COUNT)
    print("Video fps:",fps,"fps")
    
else:
    print("Video wasn't opened corretly", name)
    exit    

In [ ]:
kernel_size = 5

alpha = 0.6
beta = (1.0 - alpha)

method = False

save = False
canny_array = []
superimposed_aray = []
nai = []
ellipse_param = ()

time = 0

time_axis = np.zeros(int(total_frame))
area_axis = np.zeros(int(total_frame))
width_axis = np.zeros(int(total_frame))
height_axis = np.zeros(int(total_frame))
width_axis2 = np.zeros(int(total_frame))
height_axis2 = np.zeros(int(total_frame))
volume_axis = np.zeros(int(total_frame))
volume_axis2 = np.zeros(int(total_frame))
ellipse_area_axis = np.zeros(int(total_frame))
rev_volume_axis = np.zeros(int(total_frame))
rev_area_axis = np.zeros(int(total_frame))

ellipse_backup = np.zeros(int(total_frame),dtype=object)
rec_backup = np.zeros(int(total_frame),dtype=object)
ellipse_vertex_backup = np.zeros(int(total_frame),dtype=object)

end = False
join= False
stop = False
play = False

first = True

global skip, timer

skip = False
timer = 0

import global_
global_.ellipse_vertex = np.zeros((4,2))

In [ ]:
#Loop que passa o video frame a frame, aplicando os filtros necessarios para a aplicação de Canny
while(not end):
      
    if join == False:
      
        ret, frame = video.read()

        # Get current video time in sec 
        video_time = video.get(cv.CAP_PROP_POS_MSEC)/1000

        # Define index of next, current and previous frame
        next_frame = video.get(cv.CAP_PROP_POS_FRAMES)
        current_frame = next_frame - 1
        previous_frame = current_frame - 1
  
        # Creates a copy of unaltered frame
        cache = np.copy(frame)
        
    else:
        frame = np.copy(cache)

    if ret:

        filtered = functions.filters(frame)   # Apply filters on frame

        # Execute on first iteraction
        if first == True:
            
            # Get frame information
            frame_dimentions = frame.shape
            frame_height = frame.shape[0]
            frame_width = frame.shape[1]
            frame_channels = frame.shape[2]
            frame_size = (frame_width,frame_height)      
            
            first = False
            print(frame_size)
            
            # Create setup windows
            cv.namedWindow('Comandos', cv.WINDOW_NORMAL)
            cv.resizeWindow('Comandos', 700, 10)
            cv.createTrackbar('Frame:', 'Comandos', int(current_frame), int(total_frame), video_progression)          
            
        # Aplica o filtro binario e detecta a borda do coracao
        _ , binary_img = cv.threshold(filtered,binary_treshold,255,cv.THRESH_BINARY) # 240
        binary_edges = cv.Canny(binary_img, 50, 100, kernel_size, L2gradient= True)

        # Converte edges para opencv e sobrepoe o resultado em cima do frame original
        binary_edges_convert = cv.cvtColor(binary_img,cv.COLOR_GRAY2BGR)
        binary_superiposed = cv.addWeighted(frame, alpha, binary_edges_convert, beta, 0.0)

        # Encaixa a melhor elipse no contortno do coração
        contours, main_contour, global_.ellipse_param, rec, drawing, area, method,stop = functions.fit_ellipse(binary_img,method,join,stop,scale)
        
        if ellipse_backup[int(current_frame)] != 0:
            global_.ellipse_param = ellipse_backup[int(current_frame)]
            global_.ellipse_center = (int(ellipse_param[0][0]),int(ellipse_param[0][1]))
            global_.ellipse_size = (int(ellipse_param[1][0]),int(ellipse_param[1][1]))
            global_.ellipse_angle = int(ellipse_param[2])
            global_.ellipse_param = (global_.ellipse_center,global_.ellipse_size,global_.ellipse_angle)

            rec = rec_backup[int(current_frame)]
            global_.ellipse_vertex = ellipse_vertex_backup[int(current_frame)]

        cv.ellipse(frame, global_.ellipse_param, (90,90,255), 1)    # Draw ellipse on main frame
        
        # Abre janela com o frame original, filtrado, bordas detectadas e sobreposta em cima do frame original 
        cv.imshow('Original',frame)
        cv.imshow('Processado',binary_img)
        cv.imshow('Canny Binario Sobreposto',binary_superiposed)
        cv.imshow('Representacao',drawing)
        #cv.imshow('Informacao',info)

        #if time == 0:
            #cv.createTrackbar('Angulo:', 'Comandos', int(ellipse_param[2]), 179, ellipse_rotation)
                           
        cv.setMouseCallback('Original', modify_elipse,[frame,rec,cache,d_pixel])
               
        while True:
            key = cv.waitKey(10)

            if key != -1 or skip == True or play == True:
                break

        if play == False and skip == False:

            # Wait until user press a key that is listed on cfg
            while not(key in cfg.key_list) :
                if play == False:
                    key = cv.waitKey(0)
                else:
                    key = cv.waitKey(1)
                    if key == cfg.play:
                        play = False
                        key = 0
                        print("ENTE")
                    #time.sleep(1/fps)

            # Se o comando for "quit" ou não houver mais frames no video, entao o loop é finalizado
            if (key == cfg.close) or (not ret):
                end = True
                join = False
            elif key == cfg.play:
                play = not(play)                
                #method = True 
                #join = False
            elif key == ord('j'):
                join = True
            elif key == cfg.nxt:
                join= False
                cv.setTrackbarPos('Frame:', 'Comandos',int(next_frame))
            elif key == cfg.prv:
                join= False
                cv.setTrackbarPos('Frame:', 'Comandos',int(previous_frame))

        else:
            if not ret:
                join = False
                end = True

        if join == False:

            height,width,ellipse_area,volume = get.ellipse_paramaters(d_pixel)
            height2,width2,volume2 = get_paramaters2(d_pixel,contours,rot_axis_loc,hor_axis)     
            volume_axis2[int(current_frame)] = volume2
            height_axis2[int(current_frame)] = height2
            width_axis2[int(current_frame)] = width2
            vol_rev, area_rev = get.vol_solid_revo(contours,rot_axis_loc) 

            # Guarda os valores do tempo, area e largura
            time_axis[int(current_frame)] = video_time
            area_axis[int(current_frame)] = area*d_pixel**2
            volume_axis[int(current_frame)] = volume
            height_axis[int(current_frame)] = height
            width_axis[int(current_frame)] = width
            ellipse_area_axis[int(current_frame)] = ellipse_area
            rev_volume_axis[int(current_frame)] = vol_rev
            rev_area_axis[int(current_frame)] = area_rev
            import global_
            ellipse_backup[int(current_frame)] = global_.ellipse_center,global_.ellipse_size,global_.ellipse_angle
            rec_backup[int(current_frame)] = rec
            
            import global_
            ellipse_vertex_backup[int(current_frame)] = global_.ellipse_vertex
            
            if key == cfg.prv:
                if previous_frame >= 0:
                    video.set(cv.CAP_PROP_POS_FRAMES, previous_frame)
                    time = time - 1/fps
                else:
                    previous_frame = 0
                    time = 0
            else:
                time = time + 1/fps

            skip = False      


    else:
        break  
    
video.release()
cv.destroyAllWindows()    

del video

(320, 240)
646.7428005010956
646.7428005010956
646.430197314451
646.430197314451
646.7428005010956
646.7428005010956
646.430197314451
646.430197314451
646.7428005010956
646.7428005010956
646.7428005010956


In [ ]:
if (key == ord('q')):
    time_axis = time_axis[0:int(current_frame)]
    area_axis = area_axis[0:int(current_frame)]
    volume_axis2 = volume_axis2[0:int(current_frame)]
    height_axis2 = height_axis2[0:int(current_frame)]
    width_axis2 = width_axis2[0:int(current_frame)]
    ellipse_area_axis = ellipse_area_axis[0:int(current_frame)]
    volume_axis = volume_axis[0:int(current_frame)]
    height_axis = height_axis[0:int(current_frame)]
    width_axis = width_axis[0:int(current_frame)]
    rev_volume_axis = rev_volume_axis[0:int(current_frame)]
    rev_area_axis = rev_area_axis[0:int(current_frame)]

ellipse_vol_const = (1/6)*np.pi*np.average(height_axis)*(width_axis)**2

# Resultados

In [ ]:
results.show_results(weight,time_axis,width_axis,height_axis,width_axis2,height_axis2,
                        area_axis,rev_area_axis,ellipse_area_axis,
                        volume_axis,rev_volume_axis,ellipse_vol_const,volume_axis2,
                        detection_method)